# Import Liraries

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout

# dataset path

In [4]:
import os
os.chdir('F:\ML project\Traffic_Sign_Recognition')

# Store data, label in the list

In [5]:
data = []
labels = []
classes = 43

# Preprocess the images

In [6]:
imgs_path = "F:\ML project\Traffic_Sign_Recognition\Train"
for i in range(classes):
    img_path = os.path.join(imgs_path, str(i)) #0-42
    for img in os.listdir(img_path):
        im = Image.open(img_path + '/' + img)
        im = im.resize((30,30))
        im = np.array(im)
        data.append(im)
        labels.append(i)
print("success")        

success


# List into numpy arrays

In [7]:
data = np.array(data)
labels = np.array(labels)

In [8]:
print(data.shape, labels.shape) 

(39209, 30, 30, 3) (39209,)


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print("training shape: ",x_train.shape, y_train.shape)
print("testing shape: ",x_test.shape, y_test.shape)

training shape:  (31367, 30, 30, 3) (31367,)
testing shape:  (7842, 30, 30, 3) (7842,)


# Convert labels to onehot encoding

In [10]:
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [11]:
from tensorflow.keras.models import Sequential

# Build the CNN model

In [12]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu", input_shape=x_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation="softmax"))

In [13]:
# Compilation of the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
epochs = 15
history = model.fit(x_train, y_train, epochs=epochs, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/15
491/491 [==============================] - 27s 53ms/step - loss: 2.0721 - accuracy: 0.5123 - val_loss: 0.3473 - val_accuracy: 0.9250
Epoch 2/15
491/491 [==============================] - 26s 54ms/step - loss: 0.5126 - accuracy: 0.8563 - val_loss: 0.1446 - val_accuracy: 0.9708
Epoch 3/15
491/491 [==============================] - 26s 53ms/step - loss: 0.3013 - accuracy: 0.9142 - val_loss: 0.0729 - val_accuracy: 0.9820
Epoch 4/15
491/491 [==============================] - 27s 54ms/step - loss: 0.2236 - accuracy: 0.9372 - val_loss: 0.0582 - val_accuracy: 0.9843
Epoch 5/15
491/491 [==============================] - 26s 54ms/step - loss: 0.1856 - accuracy: 0.9487 - val_loss: 0.0498 - val_accuracy: 0.9864
Epoch 6/15
491/491 [==============================] - 27s 55ms/step - loss: 0.1660 - accuracy: 0.9553 - val_loss: 0.0700 - val_accuracy: 0.9820
Epoch 7/15
491/491 [==============================] - 29s 58ms/step - loss: 0.1715 - accuracy: 0.9533 - val_loss: 0.0457 - val_accuracy:

# Testing on test data

In [15]:
def testing(testcsv):
    y_test = pd.read_csv(testcsv)
    label = y_test["ClassId"].values
    imgs = y_test["Path"].values
    data=[]
    for img in imgs:
        image = Image.open(img)
        image = image.resize((30,30))
        data.append(np.array(image))
    x_test=np.array(data)
    return x_test,label

In [16]:
x_test, label = testing('Test.csv')

In [18]:
#y_pred = model.predict_classes(x_test)
#y_pred

y_pred = np.argmax(model.predict(x_test), axis=-1)
y_pred

array([16,  1, 38, ..., 15,  7, 10], dtype=int64)

# Accuracy with the test data

In [19]:
from sklearn.metrics import accuracy_score
print(accuracy_score(label, y_pred))

0.9510688836104513


# Save the model

In [20]:
model.save("TSC.h5")